In [1]:
#GET INTRADAY DATA
import pandas as pd
import numpy as np
import datetime
import seaborn as sn
import matplotlib.pyplot as plt
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

def GetData_Intraday(symbol):
  url = f'https://s.cafef.vn/Lich-su-giao-dich-{symbol}-6.chn#data'
  data = pd.read_html(url)
  return data

def GetSticks_Intraday(symbol):
  url = f'https://s.cafef.vn/Lich-su-giao-dich-{symbol}-6.chn#data'
  data = pd.read_html(url)
  
  df_sticks = data[len(data)-1]
  df_sticks = pd.DataFrame(df_sticks.values,columns=['Time','PriceX','Volume','Sum Volume','Percent'])

  #Tách giá
  df_sticks['Price'] = df_sticks['PriceX'].apply(lambda x: x.split(' ')[0])
  df_sticks['Price']

  df_sticks['%'] = df_sticks['PriceX'].apply(lambda x: x.split(' ')[2])
  df_sticks['%'] = df_sticks['%'].apply(lambda x: x[1:len(x)-2])

  del df_sticks['PriceX']

  #cols = ['Volume','Sum Volume','Price','%','Percent']
  cols = ['Volume','Sum Volume','Price','%']
  df_sticks[cols] = df_sticks[cols].apply(pd.to_numeric, downcast='float', errors='coerce')



  df_sticks['Time'] = df_sticks['Time'].apply(lambda x: datetime.datetime.today().strftime('%Y-%m-%d') + ' ' + str(x))
  df_sticks['Time'] = df_sticks['Time'].astype("datetime64")
  df_sticks.set_index('Time')

  df_sticks['Money'] = df_sticks['Price']*df_sticks['Volume']

  return df_sticks

def FindShark(symbol):
  df = GetSticks_Intraday(symbol=symbol)
  sum_money = np.sum(df['Money']) #.apply(lambda x: x.split(' ')[0])
  df['Shark'] = df['Money'].apply(lambda x: True if x >= (sum_money*1)/100 else False)

  df_view = df
  df_view = df_view[df_view['Shark'] == True]
  df_view['Money'] = df_view['Money'].apply("{0:,.2f}".format)  
  return df_view

In [ ]:
#BOOK VALUE

url = 'http://s.cafef.vn/hose/VND-cong-ty-co-phan-chung-khoan-vndirect.chn'